# RUN PERFORMANCE PROJECT - Pau Sampietro

## Data Ingestion, Cleaning and Export

In this file we obtain the data from suunto files (.fit), we extract the relevant information, which is stored into two dataframes:
 * Records: contains all the records from every move (suunto watches take data every 1 second) (3.000.000 rows aprox)
 * Moves: contains general information about each move. (840 rows aprox)
    
1. First, we obtain all the Data
2. Then we start cleaning null/nan values, adding missing information (accumulated ascent), etc.
3. Finally, we export all this data to a Google Cloud Database.

#### Import Libraries

In [315]:
import sys
from fitparse import FitFile
import os
import pandas as pd
import pymysql
from sqlalchemy import create_engine
import matplotlib.pyplot as plt

#### Getting Path of all running moves (locally stored in two different folders)

In [316]:
# Pau's activities
directory = 'C:/Users/Pau Sampietro/Desktop/IRONHACK/Movescount_data/Moves/'
running_files = []
for filename in os.listdir(directory):
    if filename.endswith('Running.fit') : 
        running_files.append(os.path.join(directory, filename))
        
# Armand's activities
directory = 'C:/Users/Pau Sampietro/Desktop/IRONHACK/Movescount_data/Moves_A/'
for filename in os.listdir(directory):
    if filename.endswith('Running.fit') : 
        running_files.append(os.path.join(directory, filename))
                
running_files[:5]

['C:/Users/Pau Sampietro/Desktop/IRONHACK/Movescount_data/Moves/Move_2017_07_02_11_46_40_Running.fit',
 'C:/Users/Pau Sampietro/Desktop/IRONHACK/Movescount_data/Moves/Move_2017_07_02_12_32_23_Running.fit',
 'C:/Users/Pau Sampietro/Desktop/IRONHACK/Movescount_data/Moves/Move_2017_07_19_19_49_13_Running.fit',
 'C:/Users/Pau Sampietro/Desktop/IRONHACK/Movescount_data/Moves/Move_2017_07_20_19_56_41_Running.fit',
 'C:/Users/Pau Sampietro/Desktop/IRONHACK/Movescount_data/Moves/Move_2017_07_24_18_17_34_Running.fit']

### Creating DataFrames with data of every move and every record (Tables: "moves" and "records")

In [317]:
# Initializing categories we want to store in Moves
ls_move_id = []
ls_start_time = []
ls_total_moving_time = []
ls_total_distance = []
ls_total_calories = []
ls_athlete = []
# Initializing Categories for df Records
ls_move_code = []
ls_record_code = []
ls_altitude = []
ls_cadence = []
ls_distance = []
ls_enhanced_altitude = []
ls_enhanced_speed = []
ls_heart_rate = []
ls_position_lat = []
ls_position_long = []
ls_speed = []
ls_timestamp = []
ls_vertical_speed = []

move_code = 1
for filepath in running_files:
    fitfile = FitFile(filepath)
    # Get all data messages that are of type record (information for df "records")
    record_code = 1
    for record in fitfile.get_messages('record'):
        # Go through all the data entries in this record and fill the lists
        ls_move_code.append(move_code)
        ls_record_code.append(record_code)
        ls_altitude.append(record.get_value('altitude'))
        ls_cadence.append(record.get_value('cadence'))
        ls_distance.append(record.get_value('distance'))
        ls_enhanced_altitude.append(record.get_value('enhanced_altitude'))
        ls_enhanced_speed.append(record.get_value('enhanced_speed'))
        ls_heart_rate.append(record.get_value('heart_rate'))
        ls_position_lat.append(record.get_value('position_lat'))
        ls_position_long.append(record.get_value('position_long'))
        ls_speed.append(record.get_value('speed'))
        ls_timestamp.append(record.get_value('timestamp'))
        ls_vertical_speed.append(record.get_value('vertical_speed'))
        record_code += 1
        
    # Get message data with general information (information for df "moves")
    for elem in fitfile.get_messages('session'):
        ls_move_id.append(move_code)
        ls_start_time.append(elem.get_value('start_time'))
        ls_total_moving_time.append(elem.get_value('total_moving_time'))
        ls_total_distance.append(elem.get_value('total_distance'))
        ls_total_calories.append(elem.get_value('total_calories'))
        if 'Moves_A' in filepath:
            ls_athlete.append('A')
        else:
            ls_athlete.append('P')

    move_code += 1

### 1. Creating DataFrame Moves

In [318]:
# Creating DF with all the general info of the moves
moves = pd.DataFrame({'move': ls_move_id,
                        'start_time': ls_start_time,
                        'moving_time': ls_total_moving_time,
                        'distance': ls_total_distance,
                        'calories': ls_total_calories,
                        'athlete': ls_athlete
                       })

#### Basic exploratory analysis of moves DF

In [319]:
moves.head()

,move,start_time,moving_time,distance,calories,athlete
0,1,2017-07-02 09:46:40,1787.960,4019.0,670.0,P
1,2,2017-07-02 10:32:23,1856.488,4995.0,585.0,P
2,3,2017-07-19 17:49:13,1356.937,4374.0,602.0,P
3,4,2017-07-20 17:56:41,1014.037,3005.0,365.0,P
4,5,2017-07-24 16:17:34,2522.062,6540.0,760.0,P


In [320]:
moves.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 817 entries, 0 to 816
Data columns (total 6 columns):
move           817 non-null int64
start_time     817 non-null datetime64[ns]
moving_time    817 non-null float64
distance       817 non-null float64
calories       807 non-null float64
athlete        817 non-null object
dtypes: datetime64[ns](1), float64(3), int64(1), object(1)
memory usage: 38.4+ KB


In [321]:
moves.isnull().sum()

move            0
start_time      0
moving_time     0
distance        0
calories       10
athlete         0
dtype: int64

In this case, we could see that only some "calories" values are missing, so the data is quite good to start the analysis. Anyway, we will delete the moves that have distance 0.0, assuming they are errors

In [322]:
moves = moves[moves.distance > 0.0]

### 2. Creating DataFrame Records

In [323]:
# Creating DF with all the records from all the moves
records = pd.DataFrame({'idmove': ls_move_code,
                        'idrecord': ls_record_code,
                        'latitude': ls_position_lat,
                        'longitude': ls_position_long,
                        'distance': ls_distance,
                        'altitude': ls_altitude,
                        'enhanced_altitude': ls_enhanced_altitude,
                        'cadence': ls_cadence,
                        'speed': ls_speed,
                        'enhanced_speed': ls_enhanced_speed,
                        'vertical_speed': ls_vertical_speed,
                        'heart_rate': ls_heart_rate,
                        'timestamp': ls_timestamp 
                       })

#### Basic exploratory analysis of records DF

In [324]:
records.head(3)

,idmove,idrecord,latitude,longitude,distance,altitude,enhanced_altitude,cadence,speed,enhanced_speed,vertical_speed,heart_rate,timestamp
0,1,1,495379858.0,28361936.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-07-02 09:46:42
1,1,2,495380072.0,28361805.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-07-02 09:46:43
2,1,3,495380502.0,28361626.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-07-02 09:46:45


In [325]:
records.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3095712 entries, 0 to 3095711
Data columns (total 13 columns):
idmove               int64
idrecord             int64
latitude             float64
longitude            float64
distance             float64
altitude             float64
enhanced_altitude    float64
cadence              float64
speed                float64
enhanced_speed       float64
vertical_speed       float64
heart_rate           float64
timestamp            datetime64[ns]
dtypes: datetime64[ns](1), float64(10), int64(2)
memory usage: 307.0 MB


In [326]:
len(records)

3095712

In [327]:
records.isnull().sum()

idmove                     0
idrecord                   0
latitude             1652110
longitude            1652110
distance             1659830
altitude              164350
enhanced_altitude     164350
cadence              2622810
speed                 160138
enhanced_speed        160138
vertical_speed        187749
heart_rate           1578767
timestamp                  0
dtype: int64

As we could see in the cell above, there are a lot of null values due to:
  * GPS signal lose
  * Measurement errors of the heart rate monitor 
  * Other unknown issues

1. Latitude/Longitude/Cadence are not a problem (we won't use them for the analysis)
2. Altitude/Speed do are a problem
3. Heart-rate could be filled with average values

#### Cleaning Data and adding some missing information

* Filling NaN Heart-rate values (Armand's watch) with his average heart rate

In [328]:
records['heart_rate'].fillna(150.0, inplace=True)

* Delete all the records with NaN values in Speed or Altitude categories (almost always both occur simultaneously)

In [329]:
print(f'deleted values (noise): {round(len(records[records.altitude.isnull() | (records.speed.isnull())])/len(records)*100, 2)} %')

deleted values (noise): 5.32 %


In [330]:
records.drop(records[(records.altitude.isnull() | records.speed.isnull())].index, axis=0, inplace=True)

* Fill NaN values in 'vertical_speed' with 0's

In [331]:
records['vertical_speed'].fillna(0.0, inplace=True)

* Converting speed (m/s) to pace (min/km)

In [349]:
def converting_speed(x):
    if x == 0.0:
        return x
    else:
        return round(1000 / (x * 60), 2)

In [350]:
records['pace'] = records.apply(lambda x: converting_speed(x['enhanced_speed']), axis = 1)

In [334]:
records.isnull().sum()

idmove                     0
idrecord                   0
latitude             1588003
longitude            1588003
distance             1499347
altitude                   0
enhanced_altitude          0
cadence              2462829
speed                      0
enhanced_speed             0
vertical_speed             0
heart_rate                 0
timestamp                  0
pace                       0
dtype: int64

As we could see in the cell above, there are no important missing values in the dataframe anymore, so we can proceed to calcultate the total accumulated ascent for every record (3M). This ascent will be calculated according to these rules:
 1. _When a point is higher than the previous one, we accumulate the ascent, otherwise the accumulated ascent is not modified_
 2. _When the ***idmove*** changes, we restart the accumulated ascent (because it is a different activity)_

* Applying function to add accumulated ascent to every remaining record

In [335]:
records.reset_index(drop=True, inplace=True)

In [336]:
accum_altitude = [0]
accum_alt = 0
for i in range(1, len(records)):
    if records['idmove'].iloc[i] == records['idmove'].iloc[i-1]:
        accum_alt += max(records['enhanced_altitude'].iloc[i] - records['enhanced_altitude'].iloc[i-1], 0)
    else:
        accum_alt = 0
    accum_altitude.append(accum_alt)

In [337]:
len(accum_altitude) == len(records)

True

_We get sure that we are getting the accumulated ascent for every records, in order to then do de aggregated maximum for each move_

* Adding the column to the 'records' DataFrame

In [338]:
records['accum_ascent'] = pd.Series(accum_altitude)

#### Adding some information to the "moves" DataFrame
* total ascent (according to aggregated maximum for all the records of the same move)
* times in minutes instead of seconds
* average pace in min/km (speed)

##### * Calculating and merging the total ascent for each move

In [339]:
accum_ascent = records[['idmove','accum_ascent']].groupby('idmove').max().reset_index()

In [340]:
accum_ascent.head()

,idmove,accum_ascent
0,1,280.0
1,2,63.0
2,3,115.0
3,4,91.0
4,5,241.0


In [341]:
moves = moves.merge(accum_ascent, how = 'left', left_on='move', right_on = 'idmove').drop(columns='idmove')

In [342]:
moves = moves[~ moves['accum_ascent'].isnull()]

##### * Moving time in minutes

In [343]:
moves['moving_time_min'] = moves.apply(lambda x: round(x['moving_time']/60.0, 1), axis=1)

In [344]:
moves.drop(columns='moving_time', axis = 1, inplace=True)

In [345]:
moves.rename(columns={'moving_time_min':'moving_time'}, inplace = True)

###### * Average pace during the move (speed)

In [346]:
moves['pace'] = moves.apply(lambda x: round(x['moving_time']/(x['distance']/1000), 2), axis=1)

In [347]:
moves.head()

,move,start_time,distance,calories,athlete,accum_ascent,moving_time,pace
0,1,2017-07-02 09:46:40,4019.0,670.0,P,280.0,29.8,7.41
1,2,2017-07-02 10:32:23,4995.0,585.0,P,63.0,30.9,6.19
2,3,2017-07-19 17:49:13,4374.0,602.0,P,115.0,22.6,5.17
3,4,2017-07-20 17:56:41,3005.0,365.0,P,91.0,16.9,5.62
4,5,2017-07-24 16:17:34,6540.0,760.0,P,241.0,42.0,6.42


In [351]:
records.head()

,idmove,idrecord,latitude,longitude,distance,altitude,enhanced_altitude,cadence,speed,enhanced_speed,vertical_speed,heart_rate,timestamp,pace,accum_ascent
0,1,6,495381218.0,28361471.0,14.0,3165.0,133.0,90.0,2100.0,2.1,0.0,110.0,2017-07-02 09:46:48,7.94,0.0
1,1,7,495381528.0,28361399.0,17.0,3165.0,133.0,90.0,2100.0,2.1,0.0,110.0,2017-07-02 09:46:49,7.94,0.0
2,1,8,495381826.0,28361339.0,20.0,3165.0,133.0,90.0,2500.0,2.5,0.0,110.0,2017-07-02 09:46:50,6.67,0.0
3,1,9,495382113.0,28361268.0,22.0,3170.0,134.0,90.0,2500.0,2.5,0.0,110.0,2017-07-02 09:46:51,6.67,1.0
4,1,10,495382411.0,28361184.0,25.0,3170.0,134.0,90.0,2600.0,2.6,0.0,110.0,2017-07-02 09:46:52,6.41,1.0


### Exporting both DataFrames ("records" and "moves") to Google Cloud DB

* For this purpose, we have created a library (rpdb) with some methods to import and export data from/to the database.

In [359]:
from rpdb import read_table, export_table

In [360]:
# def export_table(dataframe, table, METHOD='replace')

In [361]:
export_table(moves, table='moves')

In [362]:
export_table(records, table='records')